In [1]:
import pandas as pd
df = pd.read_csv("../data/raw/RipleyChile_2022-12-27.csv")
df.count()

user_id          450
user_location    191
created_at       450
tweet_text       450
dtype: int64

In [2]:
# eliminación de textos repetidos.
df.drop_duplicates(subset=['tweet_text'], inplace=True)
df.count()

user_id          412
user_location    175
created_at       412
tweet_text       412
dtype: int64

In [3]:
import re
text = "@RipleyChile #ojopollo OJO, así vienen los productos de ripley… uno por mejor comprando los regalos de navidad en su tienda y los productos vienen en estas condiciones https://t.co/GqljLK8BAO"
text = re.sub(r'https?:\/\/\S*', '', text, flags=re.MULTILINE) #elimina el link t.co
text = re.sub(r'@\S*', '', text) # eliminación de cuentas
text = re.sub(r'#\S*', '', text) # eliminación hashtags
text

'  OJO, así vienen los productos de ripley… uno por mejor comprando los regalos de navidad en su tienda y los productos vienen en estas condiciones '

In [4]:
# eliminar los nick o username de las tiendad en los textos.
import re
from cleantext import clean
dic = {
    "@usernames":[],
    "hashtags": [],
}
for row in df.iterrows():
    in_text = row[1]["tweet_text"]
    if re.findall(r"@(\w+)", in_text):
        dic["@usernames"].append(re.findall(r"@(\w+)", in_text))
    else:
        dic["@usernames"].append(float("nan"))
    if re.findall(r"#(\W+)", in_text):
        dic["hashtags"].append(re.findall(r"#(\W+)", in_text))
    else:
        dic["hashtags"].append(float("nan"))
    out_text = in_text.lower() # se convienten las letras mayusculas a minusculas
    out_text = clean(out_text, no_emoji=True)
    out_text = re.sub(r'https?:\/\/\S*', '', out_text, flags=re.MULTILINE) #elimina el link t.co
    out_text = re.sub(r'@\S*', '', out_text) # eliminación de cuentas
    out_text = re.sub(r'#\S*', '', out_text) # eliminación hashtags
    df["tweet_text"].replace([in_text], out_text, inplace=True)
df["@usernames_in_tweet"] = dic["@usernames"]
df["hashtags_in_tweet"] = dic["hashtags"]
df.head(2)

,user_id,user_location,created_at,tweet_text,@usernames_in_tweet,hashtags_in_tweet
0,1071459919306592256,NaN,2022-12-27 15:31:31+00:00,buenas tardes ?como recupero una boleta si no...,[RipleyChile],NaN
1,851487655875489793,NaN,2022-12-27 15:30:05+00:00,no han respondido mis consultas,[RipleyChile],NaN


In [5]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

model_path = "daveni/twitter-xlm-roberta-emotion-es"
tokenizer = AutoTokenizer.from_pretrained(model_path )
config = AutoConfig.from_pretrained(model_path )
# PT
model = AutoModelForSequenceClassification.from_pretrained(model_path)

#df[list(config.id2label.values())]=0
dic = {}
for key in config.id2label.values():
    dic[key] = []

for row in df.iterrows():
    text = row[1]["tweet_text"]
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    for key in config.id2label.keys():
        dic[config.id2label[key]].append(scores[key])

for key in config.id2label.values():
    df[key] = dic[key]

df.head(3)

/home/unknownsystem/Public/tt-pregrado/env_prueba/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,user_id,user_location,created_at,tweet_text,@usernames_in_tweet,hashtags_in_tweet,sadness,joy,anger,surprise,disgust,fear,others
0,1071459919306592256,NaN,2022-12-27 15:31:31+00:00,buenas tardes ?como recupero una boleta si no...,[RipleyChile],NaN,0.142813,0.037846,0.090811,0.047647,0.014096,0.014696,0.652092
1,851487655875489793,NaN,2022-12-27 15:30:05+00:00,no han respondido mis consultas,[RipleyChile],NaN,0.125220,0.080856,0.144959,0.061590,0.033295,0.026135,0.527947
2,97794104,Concepción,2022-12-27 15:07:57+00:00,el dia 19 de diciembre realice una compra en ...,[RipleyChile],NaN,0.107856,0.040425,0.051369,0.047153,0.013840,0.016875,0.722484


In [7]:
from transformers import pipeline
model_path = "daveni/twitter-xlm-roberta-emotion-es"
emotion_analysis = pipeline("text-classification", framework="pt", model=model_path, tokenizer=model_path)
emotion_analysis("Einstein dijo: Solo hay dos cosas infinitas, el universo y los pinches anuncios de bitcoin en Twitter. Paren ya carajo aaaaaaghhgggghhh me quiero murir")[0]

{'label': 'anger', 'score': 0.4830705523490906}